In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
from kaggle_secrets import UserSecretsClient
import wandb

user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("wandb-key")

wandb.login(key=secret_value_1)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [14]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
import seaborn as sns
sns.set_style("whitegrid")
import torchaudio
from torch.utils.data import DataLoader, Dataset
import random
import os
from IPython import display
from tqdm import tqdm
import csv
import wandb 

In [15]:
def save_predictions(predictions, filename="predictions.csv"):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for file_id, score in predictions.items():
            writer.writerow([file_id, f"{score:.5f}"])

In [30]:
epochs = 2 # Вынесено в отдельную переменную для тестов
runs = 1 # В training recipe было указано 6, но, как я понял, нам это делать и сравнивать результаты не нужно. На всякий случай вынесено в отдельную переменную.

In [17]:
class MyDataset(Dataset):
    def __init__(self, data_type="train"):
        self.files = []
        self.data_type = data_type
        self.path = os.path.join("/kaggle/input/asvspoof2019-la", "LA", f"ASVspoof2019_LA_{data_type}/flac")
        
        if data_type == "train": # Не понял почему, но в train папке данный файл по-другому называется, поэтому нужно выделять отдельный случай
             name = "ASVspoof2019.LA.cm.train.trn.txt"
        else:
            name = f"ASVspoof2019.LA.cm.{data_type}.trl.txt"
        
        self.protocol_file = os.path.join("/kaggle/input/asvspoof2019-la", "LA", "ASVspoof2019_LA_cm_protocols", name)

        files = []
        with open(self.protocol_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                file_name = parts[1] + '.flac'
                if parts[4] == "spoof":
                    val = 0
                else:
                    val = 1
                files.append((file_name, val))
        self.files = files

    def __getitem__(self, index):
        file_name, label = self.files[index]
        file_path = os.path.join(self.path, file_name)

        inpt, sr = torchaudio.load(file_path)
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(sr, 16000)
            inpt = resampler(inpt)

        spectrogram = self.FFT(inpt)
        base_name = file_name.replace('.flac', '')
        return spectrogram, label, base_name
    
    def FFT(self, inpt):
        spec = torch.stft(input=inpt, n_fft=2048, hop_length=512, win_length=2048, window=torch.hann_window(2048), return_complex=True) # Используем готовый SFTF
        spec = torch.log(torch.abs(spec) + 1e-9) # Логарифм как в статье

        # Придаем нужную форму, чтобы при выполнении не выдало ошибку
        
        _, freq, time = spec.shape
        if freq > 864:
            spec = spec[:, :864, :]
        else:
            spec = torch.nn.functional.pad(spec, (0, 0, 0, 864 - freq))
        if time > 600:
            spec = spec[:, :, :600]
        else:
            spec = torch.nn.functional.pad(spec, (0, 600 - time, 0, 0))
        
        return spec

    def __len__(self):
        return len(self.files)

In [18]:
class MFM(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        halves = torch.split(x, x.size(1)//2, dim=1)
        return torch.max(halves[0], halves[1])

class MyModel(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2, bias=False),
            MFM(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=1, stride=1, padding=0, bias=False),
            MFM(),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 96, kernel_size=3, stride=1, padding=1, bias=False),
            MFM(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(48),
            
            nn.Conv2d(48, 96, kernel_size=1, stride=1, padding=0, bias=False),
            MFM(),
            nn.BatchNorm2d(48),
            nn.Conv2d(48, 128, kernel_size=3, stride=1, padding=1, bias=False),
            MFM(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=1, stride=1, padding=0, bias=False),
            MFM(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False),
            MFM(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=1, stride=1, padding=0, bias=False),
            MFM(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            MFM(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(),
            
            nn.Linear(32 * 54 * 37, 160),
            MFM(),
            nn.BatchNorm1d(80),
            nn.Dropout(p=0.5), # Т.к. сказали добавить dropout
            nn.Linear(80, 2)
        )


    def forward(self, input_data):
        return self.net(input_data)

In [19]:
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    print(f"Seed: {seed}")

In [31]:
def train_one_epoch(model, dataloader, criterion, optimizer, 
                    device, metrics_logs, total_steps, curr_epoch):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    avg_loss = 0.0
    
    for batch_idx, (inpt, label, file_id) in tqdm(enumerate(dataloader), total=len(dataloader), desc="Training"):
        # ОСНОВНАЯ ЧАСТЬ
        
        inpt, label = inpt.to(device), label.to(device)

        output = model(inpt)
        loss = criterion(output, label)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # СЧИТАЕМ/ИЗМЕНЯЕМ ПЕРЕМЕННЫЕ ДЛЯ МЕТРИК
        
        epoch_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        step_correct = (predicted == label).sum().item()
        step_total = label.size(0)
        step_acc = 100 * step_correct / step_total
        total_steps += 1

        # СОХРАНЯЕМ МЕТРИКИ
        
        wandb.log({"train loss vs step": loss.item(),
                   "train accuracy vs step": step_acc, "train epoch vs step": curr_epoch
        }, step=total_steps)

    avg_loss = epoch_loss / (batch_idx + 1)
    epoch_acc = 100 * correct / total
    return avg_loss, epoch_acc, total_steps

def evaluate(model, dataloader, criterion, device, return_predictions=False, curr_step=None):
    model.eval()
    epoch_loss = 0.0
    correct = 0
    total = 0
    avg_loss = 0
    answs = []
    prdns = []
    file_ids = []
    predictions_dict = {} 
    
    for batch_idx, (inpt, label, file_id) in tqdm(enumerate(dataloader), total=len(dataloader), desc="Evaluating"):
        inpt, label = inpt.to(device), label.to(device)
        
        with torch.no_grad():
            # ОСНОВНАЯ ЧАСТЬ
            
            output = model(inpt)
            loss = criterion(output, label)

            # СНОВА КУЧА ПЕРЕМЕННЫХ ДЛЯ МЕТРИК
            
            epoch_loss += loss.item()
            _, predicted = torch.max(output, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            
            scores = torch.softmax(output, dim=1)[:, 1]
            answs.extend(label.cpu().numpy())
            prdns.extend(scores.cpu().numpy())
            file_ids.extend(file_id)
    
    avg_loss = epoch_loss / (batch_idx + 1)
    accuracy = 100 * correct / total

    # Считаем EER
    bonafide_scores = np.array([prdns[i] for i in range(len(answs)) if answs[i] == 1])
    spoof_scores = np.array([prdns[i] for i in range(len(answs)) if answs[i] == 0])
    eer, _ = compute_eer(bonafide_scores, spoof_scores)
    eer *= 100
    
    if return_predictions: # Для финальной оценки модели на eval датасете
        # Записываем предсказания
        for i, file_id in enumerate(file_ids):
            predictions_dict[file_id] = prdns[i]
            
        # Сохраняем данные в wandb
        wandb.log({"eval loss": avg_loss, "eval accuracy": accuracy, "eval eer": eer})
        
        return avg_loss, accuracy, eer, predictions_dict
    else: # Для запусков evaluate после тренировки каждой эпохи на dev датасете
        # Записываем предсказания (Только для финальной версии)
        for i, file_id in enumerate(file_ids):
            predictions_dict[file_id] = prdns[i]
        csv_filename = f"predictions_{curr_step}.csv"
        save_predictions(predictions_dict, csv_filename)
        print(f"Predictions: {csv_filename}")
        
        wandb.log({"eval loss vs step": avg_loss, "eval accuracy vs step": accuracy,
            "eval eer vs step": eer}, step=curr_step)
        return avg_loss, accuracy, eer

def train(model, train_dataloader, dev_dataloader, optimizer, criterion, scheduler, device, epochs, metrics_logs, total_steps):

    epoch = 0
    while epoch < epochs:
        train_loss, train_acc, total_steps = train_one_epoch(model, train_dataloader, criterion, 
                                                             optimizer, device, metrics_logs, total_steps, curr_epoch=epoch)

        
        if (epoch + 1) % 2 == 0 or epoch == epochs - 1 or epoch < 3: 
            # Говорили делать val через две эпохи, но у меня тогда мало данных для графиков
            val_loss, val_acc, val_eer = evaluate(model, dev_dataloader, criterion, device,
                                                  curr_step=total_steps)
        else:
            val_loss, val_acc, val_eer = None, None, None
        
        scheduler.step()
        
        #Выводим логи
        log_str = f"Epoch {epoch+1}: "
        log_str += f"train loss = {train_loss:.4f} train acc = {train_acc:.2f}% "
        if val_acc is not None:
            log_str += f"eval loss = {val_loss:.4f} eval acc = {val_acc:.2f}% eval eer = {val_eer:.2f}%"
        print(log_str)

        if epoch == epochs - 1:
            print("Continue? (1 or 0)")
            ans_count = int(input())
            epochs += ans_count

        epoch += 1
    
    return model, metrics_logs

In [21]:
def compute_det_curve(target_scores, nontarget_scores):

    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate(
        (np.ones(target_scores.size), np.zeros(nontarget_scores.size)))

    # Sort labels based on scores
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]

    # Compute false rejection and false acceptance rates
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - \
        (np.arange(1, n_scores + 1) - tar_trial_sums)

    # false rejection rates
    frr = np.concatenate(
        (np.atleast_1d(0), tar_trial_sums / target_scores.size))
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums /
                          nontarget_scores.size))  # false acceptance rates
    # Thresholds are the sorted scores
    thresholds = np.concatenate(
        (np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))

    return frr, far, thresholds


def compute_eer(bonafide_scores, other_scores):
    """ 
    Returns equal error rate (EER) and the corresponding threshold.
    """
    frr, far, thresholds = compute_det_curve(bonafide_scores, other_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]

In [32]:
wandb.init(
    project="ASVspoof2019_final",
    config={"learning_rate": 3e-4, "batch_size": 8,
            "epochs": epochs, "architecture": "LCNN"}
)
    
device = "cuda" if torch.cuda.is_available() else "cpu"
results = []
eval_results = []
    
for run in range(1, runs + 1):
    # Изначально использовался метод получения seed из статьи
    # seed = 10 ** (run - 1)
    seed = random.randint(1, 100000)
    #seed = 96498
    set_random_seed(seed)

    # Параметры для локальных графиков, в финальной версии не используются
    metrics_logs = {'train_acc_by_step': [], 'train_loss_by_step': [], 'val_loss_by_step': [], 'val_acc_by_step': [], 'val_eer_by_step': [],
                'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_eer': []}
    total_steps = 0

    # ОБУЧЕНИЕ
        
    model = MyModel().to(device)
    train_dataset = MyDataset(data_type="train")
    dev_dataset = MyDataset(data_type="dev")
    eval_dataset = MyDataset(data_type="eval")

    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, pin_memory=True)
    dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False)
    eval_dataloader = DataLoader(eval_dataset, batch_size=8, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999), eps=1e-8) 
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    # Параметры выше взяты из статьи с Training recipe
    
    trained_model, run_logs = train(
        model=model,
        train_dataloader=train_dataloader,
        dev_dataloader=eval_dataloader,
        optimizer=optimizer,
        criterion=criterion,
        scheduler=scheduler,
        device=device,
        epochs=epochs,
        metrics_logs=metrics_logs,
        total_steps=total_steps
    )

wandb.log({
    "seed": seed
})

wandb.finish()

Seed: 94097


Evaluating:  26%|██▌       | 2313/8905 [02:52<08:10, 13.43it/s]


KeyboardInterrupt: 

In [24]:
"""
# ФИНАЛЬНАЯ ОЦЕНКА МОДЕЛИ НА EVAL
        
eval_loss, eval_acc, eval_eer, eval_predictions  = evaluate(trained_model, eval_loader, criterion, device,
                                                return_predictions=True)

# ЛОГИ И ВЫВОД РЕЗУЛЬТАТОВ
        
wandb.log({
    "eval loss": eval_loss,
    "eval accuracy": eval_acc,
    "eval eer": eval_eer,
    "seed": seed
})
csv_filename = f"predictions.csv"
save_predictions(eval_predictions, csv_filename)
print(f"Predictions: {csv_filename}")
        
print(f"Results: loss = {eval_loss:.4f}, acc = {eval_acc:.4f}%, eer = {eval_eer:.4f}%")

results.append(run_logs)
eval_results.append({'loss': eval_loss, 'acc': eval_acc, 'eer': eval_eer})

wandb.finish()
"""

'\n# ФИНАЛЬНАЯ ОЦЕНКА МОДЕЛИ НА EVAL\n        \neval_loss, eval_acc, eval_eer, eval_predictions  = evaluate(trained_model, eval_loader, criterion, device,\n                                                return_predictions=True)\n\n# ЛОГИ И ВЫВОД РЕЗУЛЬТАТОВ\n        \nwandb.log({\n    "eval loss": eval_loss,\n    "eval accuracy": eval_acc,\n    "eval eer": eval_eer,\n    "seed": seed\n})\ncsv_filename = f"predictions.csv"\nsave_predictions(eval_predictions, csv_filename)\nprint(f"Predictions: {csv_filename}")\n        \nprint(f"Results: loss = {eval_loss:.4f}, acc = {eval_acc:.4f}%, eer = {eval_eer:.4f}%")\n\nresults.append(run_logs)\neval_results.append({\'loss\': eval_loss, \'acc\': eval_acc, \'eer\': eval_eer})\n\nwandb.finish()\n'

In [25]:
"""
def test_eer():
    bonafide = np.random.normal(0.8, 0.1, 1000)  # Низкие значения
    spoof = np.random.normal(0.2, 0.1, 1000)      # Высокие значения
    eer, _ = compute_eer(bonafide, spoof)
    print(f"Expected low EER, got: {eer:.4f}")

    bonafide = np.random.uniform(0, 1, 1000)
    spoof = np.random.uniform(0, 1, 1000)
    eer, _ = compute_eer(bonafide, spoof)
    print(f"Expected ~0.5 EER, got: {eer:.4f}")

test_eer()
"""

'\ndef test_eer():\n    bonafide = np.random.normal(0.8, 0.1, 1000)  # Низкие значения\n    spoof = np.random.normal(0.2, 0.1, 1000)      # Высокие значения\n    eer, _ = compute_eer(bonafide, spoof)\n    print(f"Expected low EER, got: {eer:.4f}")\n\n    bonafide = np.random.uniform(0, 1, 1000)\n    spoof = np.random.uniform(0, 1, 1000)\n    eer, _ = compute_eer(bonafide, spoof)\n    print(f"Expected ~0.5 EER, got: {eer:.4f}")\n\ntest_eer()\n'

In [26]:


print("Содержимое рабочей директории:")
print(os.listdir('/kaggle/working/'))


Содержимое рабочей директории:
['predictions_9519.csv', 'predictions_6346.csv', 'wandb', 'predictions_3173.csv', '.virtual_documents']


In [27]:

from IPython.display import FileLink
FileLink('predictions_6346.csv')


/kaggle/working/predictions_6346.csv

In [28]:
"""
from IPython.display import Image, display

# Для step_metrics.png
display(Image(filename='/kaggle/working/step_metrics.png', width=1000))

# Для training_results.png
display(Image(filename='/kaggle/working/training_results.png', width=1000))
"""

"\nfrom IPython.display import Image, display\n\n# Для step_metrics.png\ndisplay(Image(filename='/kaggle/working/step_metrics.png', width=1000))\n\n# Для training_results.png\ndisplay(Image(filename='/kaggle/working/training_results.png', width=1000))\n"